<a href="https://colab.research.google.com/github/RMoulla/MLW_Mars/blob/main/MLOPS_TP_BERT_Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modèle BERT pour l'analyse de sentiments**

# Objectif du TP

L’objectif de ce TP est de réaliser un fine-tuning d’un modèle BERT pré-entraîné pour la classification de commentaires sur des critiques de films (dataset IMDB). Concrètement, nous allons ajouter une couche linéaire au modèle BERT pré-entraîné, dont nous allons ajuster les poids sur un ensemble des données IMDB.

Le TP se décline selon les étapes suivantes :

# Étapes principales du TP

1. **Charger un sous-ensemble du dataset IMDB** et séparer les données en jeu d’entraînement et jeu de test.
2. **Convertir les critiques textuelles en séquences de tokens** grâce au tokenizer de BERT.
3. **Créer une classe `IMDBDataset`** qui encapsule les textes, les labels et la logique d’encodage (tokenisation et gestion des masques).
4. **Définir un modèle `BertClassifier`** composé d’un BERT pré-entraîné (gelé dans ce code) et d’une couche linéaire assurant la classification binaire.
5. **Mettre en place la procédure d’entraînement** : pour plusieurs époques successives, parcourir les données d’entraînement par batch, calculer la perte et mettre à jour les paramètres de la couche de classification.
6. **Évaluer périodiquement les performances** sur le jeu de test (calcul de la perte moyenne et de l’accuracy).

In [ ]:

!pip install torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install datasets

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:

import torch
print("PyTorch:", torch.__version__)
print("CUDA:", torch.version.cuda)
!nvcc --version

PyTorch: 2.6.0+cu118
CUDA: 11.8
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!nvidia-smi

Fri Mar 21 12:13:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   66C    P0             28W /   70W |     724MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----


## Préparation des données

Nous définissons ici la classe IMDBDataset, héritée de Dataset. Elle prend en entrée des listes de textes et de labels binaires, ainsi qu’un tokenizer BERT et une taille maximale de séquence. Dans le constructeur, on vérifie que tous les labels sont bien à 0 ou 1, puis on enregistre le tout comme attributs internes. La méthode `__getitem__` réalise la tokenisation de chaque texte, avec un éventuel tronquage ou du padding pour atteindre la longueur requise. Elle renvoie finalement un dictionnaire contenant les tenseurs `input_ids`, `attention_mask` et `labels`, prêts à être utilisés par un modèle BERT lors de l’entraînement ou de l’inférence.

In [ ]:
from torch.utils.data import Dataset
from datasets import load_dataset

class IMDBDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts # les commentaires
        self.labels = labels  # labels 0 ou 1
        self.tokenizer = tokenizer
        self.max_len = max_len

        # Vérification des labels
        unique_labels = set(self.labels)
        print(f"Labels uniques dans le dataset: {unique_labels}")
        # Comme c'est de la classification binaire, on veut {0, 1}
        assert all(lbl in [0, 1] for lbl in unique_labels), \
            f"Labels hors limite détectés: {unique_labels}"

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
print("Chargement du dataset IMDB...")
dataset = load_dataset("imdb")

Chargement du dataset IMDB...


In [ ]:
# Afficher la taille des splits
print(f"Taille du train : {len(dataset['train'])}")
print(f"Taille du test  : {len(dataset['test'])}")

Taille du train : 25000
Taille du test  : 25000


In [ ]:
dataset['train']['text'][1]

'"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn\'t matter what one\'s political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn\'t true. I\'ve seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don\'t exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we\'re treated to the site of Vincent Gallo\'s throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, and the s

In [ ]:
dataset['train']['label'][1]

0

In [ ]:

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sample = IMDBDataset(dataset['train']['text'][:1], dataset['train']['label'][:1], tokenizer, max_len=512)[0]

print("Texte original :")
print(dataset['train']['text'][1])
print("\nTokenized input :")
print(tokenizer.tokenize(dataset['train']['text'][1]))

print("\nEncodage final (input_ids) :")
print(sample["input_ids"])  # IDs des tokens
print("\nAttention mask :")
print(sample["attention_mask"])  # 1s et 0s
print("\nLabel associé :")
print(sample["labels"])  # Label (0 ou 1)

Labels uniques dans le dataset: {0}
Texte original :
"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn't true. I've seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don't exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we're treated to the site of Vincent Gallo's throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on d

## Construction et entraînement du modèle de classification

Nous allons construire le modèle de classification de sentiments sur un sous-ensemble du dataset IMDB à l’aide d’un modèle `BertClassifier`. Il définit notamment deux fonctions d’entraînement et d’évaluation, configure les hyperparamètres, crée les jeux de données et exécute plusieurs époques d’apprentissage pour optimiser la couche de classification.

- **Définition du modèle** : La classe `BertClassifier` combine un modèle BERT pré-entraîné et une couche linéaire pour la classification.

- **Fonction d’entraînement (`train_epoch`)** : Elle parcourt les données par batch, calcule la perte (cross-entropy), met à jour la couche de classification et retourne la perte moyenne et l’accuracy.

- **Fonction d’évaluation (`eval_model`)** : Elle parcourt les données de validation ou de test, calcule la perte moyenne, l’accuracy et retourne ces métriques sans modifier le modèle.

- **Configuration des hyperparamètres** : On définit ici les hyperparamètres du modèle :batch size, taux d’apprentissage, nombre d’époques, taille maximale des séquences et  nombre de classes.

- **Chargement du dataset IMDB** : On sélectionne les échantillons d'entraînement et de test.

- **Tokenisation** : On utilise `BertTokenizer` pour transformer les critiques textuelles en séquences de tokens.

- **Création des `IMDBDataset`** : On convertit des tokens et labels en objets PyTorch (input_ids, attention_mask, labels).

- **Initialisation du modèle** : Instanciation de `BertClassifier` et association au device (CPU ou GPU).

- **Boucle d’entraînement** : Pour chaque époque, on entraîne le modèle sur les données d’entraînement puis on l’évalue sur les données de validation.

In [ ]:
import torch
from torch import nn
from transformers import BertModel, BertTokenizer, AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np

# Vérifier si un GPU est disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilisation de : {device}")

######## Définition du modèle ##########

class BertClassifier(nn.Module):
    def __init__(self, n_classes):
        super(BertClassifier, self).__init__()
        # On charge BERT (base uncased)
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(p=0.3)

        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        # Retour par défaut : dictionnaire (return_dict=True)
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # Récupération du "pooler_output"
        pooled_output = outputs.pooler_output  # On récupère le token [CLS]
        output = self.drop(pooled_output)
        return self.out(output)


#### Définition de la fonction train ####


def train_epoch(model, data_loader, optimizer, device):
    # Geler les paramètres de BERT
    for param in model.bert.parameters():
        param.requires_grad = False

    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch in tqdm(data_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)

        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct_predictions += torch.sum(preds == labels)
        total_predictions += labels.size(0)

    return total_loss / len(data_loader), correct_predictions.double() / total_predictions


#### Définition de la fonction eval ####

def eval_model(model, data_loader, device):
    """
    Évalue les performances du modèle sur un jeu de validation ou de test.

    :param model: Modèle BERT ou GPT fine-tuné.
    :param data_loader: DataLoader contenant les données à évaluer.
    :param device: Appareil à utiliser (CPU ou GPU).
    :return: Moyenne de la perte et précision sur les données évaluées.
    """
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    # Désactiver la dérivation automatique
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Calcul des prédictions
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)

            # Calcul de la perte
            loss = nn.CrossEntropyLoss()(outputs, labels)
            total_loss += loss.item()

            # Calcul des prédictions correctes
            correct_predictions += torch.sum(preds == labels)
            total_predictions += labels.size(0)

    # Moyenne des pertes et précision totale
    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions.double() / total_predictions
    return avg_loss, accuracy


######## Entraînement du modèle #########



BATCH_SIZE = 16
EPOCHS = 5
MAX_LEN = 128
LEARNING_RATE = 2e-4
N_CLASSES = 2  # Binaire : positif (1) ou négatif (0)


# On prend seulement une partie du dataset pour ce test
# - Ex : 10000 pour l'entraînement
# - Ex : 500 pour le test
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
test_texts = dataset['test']['text']
test_labels = dataset['test']['label']

print(f"Taille du dataset d'entraînement : {len(train_texts)}")
print(f"Taille du dataset de test : {len(test_texts)}")

print("Distribution des labels (train):", np.bincount(train_labels))
print("Distribution des labels (test) :", np.bincount(test_labels))

print("Initialisation du tokenizer...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print("Création des datasets...")
train_dataset = IMDBDataset(train_texts, train_labels, tokenizer, max_len=MAX_LEN)
val_dataset = IMDBDataset(test_texts, test_labels, tokenizer, max_len=MAX_LEN)

print("Création des dataloaders...")
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

print("Initialisation du modèle...")
model = BertClassifier(N_CLASSES).to(device)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
best_accuracy = 0

print("Début de l'entraînement...")
for epoch in range(EPOCHS):
    print(f'\nEpoch {epoch + 1}/{EPOCHS}')

    train_loss, train_acc = train_epoch(model, train_loader, optimizer, device)
    print(f'Train loss: {train_loss:.3f}, accuracy: {train_acc:.3f}')

    val_loss, val_acc = eval_model(model, val_loader, device)
    print(f'Val loss: {val_loss:.3f}, accuracy: {val_acc:.3f}')

    # Sauvegarde du meilleur modèle si l'accuracy s'améliore
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        torch.save(model.state_dict(), 'best_model_imdb.pt')
        print(f'Nouveau meilleur modèle sauvegardé avec accuracy: {val_acc:.3f}')

    print('-' * 50)